## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
from keras.applications import ResNet50

from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                  input_tensor=input_tensor,pooling=None, classes=10)
'''Resnet 50 架構'''
#model=ResNet50(include_top=False, weights='imagenet',
               #input_tensor=input_tensor,
               #pooling=None, classes=10)
model.summary()


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
_________

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
x = Convolution2D(32, (1,1) ,activation='relu')(x)
x = MaxPooling2D(pool_size=(1,1))(x)
x = Convolution2D(64, (1,1) ,activation='relu')(x)
x = Flatten()(x)
x = Dense(units=100,activation='relu')(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 138


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  # Remove the CWD from sys.path while we load stuff.


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=30)


Epoch 1/30
50000/50000 [==============================] - 96s 2ms/step - loss: 1.4050 - accuracy: 0.5079
Epoch 2/30
50000/50000 [==============================] - 90s 2ms/step - loss: 1.0539 - accuracy: 0.6398
Epoch 3/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.9300 - accuracy: 0.6827
Epoch 4/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.8441 - accuracy: 0.7120
Epoch 5/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.7723 - accuracy: 0.7350
Epoch 6/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.7005 - accuracy: 0.7605
Epoch 7/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.6418 - accuracy: 0.7806
Epoch 8/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.5882 - accuracy: 0.8004
Epoch 9/30
50000/50000 [==============================] - 90s 2ms/step - loss: 0.5339 - accuracy: 0.8177
Epoch 10/30
50000/50000 [=============================